In [22]:
!pip install vecstack

In [0]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from collections import Counter #for Smote, 
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings("ignore")

In [24]:
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive

%cd /gdrive


trainfile = r'/gdrive/My Drive/CIS508/Assignment-3/RevisedHomesiteTrain.csv'
train_data = pd.read_csv(trainfile)

testfile = r'/gdrive/My Drive/CIS508/Assignment-3/RevisedHomesiteTest.csv'
test_data = pd.read_csv(testfile)

print(train_data.shape)
print(train_data.dtypes)
train_data.head()

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive
(65000, 596)
CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
QuoteConversion_Flag    int64
Length: 596, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,QuoteConversion_Flag
0,2,1,17,23,17,23,15,22,16,22,13,22,13,23,965.0,1.0200,23,0.9403,0.0006,-1,25,1,1,1,1,1,1,-1,7,1,2,1,2,13,22,-1,12,3,6,3,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0,0
1,5,9,6,8,6,8,5,7,5,8,13,22,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,8,...,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
2,4,6,7,12,7,12,6,10,7,11,25,25,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,11,9,15,5,12,-1,12,10,16,6,14,2,13,-1,17,21,21,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0,0
3,15,23,3,2,3,2,2,2,3,2,13,22,13,23,1165.0,1.2665,10,0.9769,0.0004,-1,25,15,22,21,22,15,22,-1,15,24,23,9,18,2,3,-1,6,10,13,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1,0
4,4,6,8,13,8,13,7,11,7,13,13,22,13,23,1487.0,1.3045,23,0.9472,0.0006,-1,25,9,17,12,16,9,17,-1,22,9,8,10,20,2,6,-1,4,17,16,2,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0


In [25]:
print(test_data.shape)
print(test_data.dtypes)
test_data.head()

(173836, 596)
CoverageField11A         int64
CoverageField11B         int64
CoverageField1A          int64
CoverageField1B          int64
CoverageField2A          int64
                         ...  
GeographicField64_CA     int64
GeographicField64_IL     int64
GeographicField64_NJ     int64
GeographicField64_TX     int64
GeographicField64       object
Length: 596, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,GeographicField64
0,13,22,4,4,4,4,3,3,3,4,13,22,13,23,1487.0,1.3045,16,0.9364,0.0006,-1,25,9,18,13,18,9,18,-1,19,10,13,8,17,2,10,-1,20,19,18,25,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,IL
1,4,5,8,14,8,14,7,12,8,13,13,22,13,23,564.0,1.1886,11,0.9919,0.0038,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,17,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
2,3,3,11,18,11,18,10,16,10,18,13,22,13,23,564.0,1.0670,15,0.8945,0.0038,-1,25,4,10,10,16,5,10,-1,11,11,18,5,13,2,13,-1,15,21,20,3,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
3,5,9,14,22,15,22,13,20,22,25,13,22,13,23,1113.0,1.2665,21,0.8870,0.0004,-1,25,24,24,23,23,24,24,25,25,23,22,24,25,2,7,-1,13,11,14,9,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,TX
4,12,21,4,5,4,5,4,4,4,5,13,22,13,23,935.0,1.0200,25,0.9153,0.0007,-1,25,2,3,3,5,2,5,-1,7,6,6,2,6,15,23,-1,17,4,8,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,CA


In [26]:
Xtrain=train_data.iloc[:,:-1].copy()
Ytrain=train_data.iloc[:,-1].copy()

Xtest=test_data.iloc[:,:-1].copy()

print(Xtrain.shape)
print(Xtrain.dtypes)
Xtrain.head()

(65000, 595)
CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField63_Y     int64
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
Length: 595, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_Q,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX
0,2,1,17,23,17,23,15,22,16,22,13,22,13,23,965.0,1.0200,23,0.9403,0.0006,-1,25,1,1,1,1,1,1,-1,7,1,2,1,2,13,22,-1,12,3,6,3,...,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,1,0,0,0
1,5,9,6,8,6,8,5,7,5,8,13,22,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,8,...,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0
2,4,6,7,12,7,12,6,10,7,11,25,25,13,23,548.0,1.2433,7,1.0006,0.0040,-1,25,4,11,9,15,5,12,-1,12,10,16,6,14,2,13,-1,17,21,21,2,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,1,0,0,1,0,1,0,0,1,0,0,0,1,0
3,15,23,3,2,3,2,2,2,3,2,13,22,13,23,1165.0,1.2665,10,0.9769,0.0004,-1,25,15,22,21,22,15,22,-1,15,24,23,9,18,2,3,-1,6,10,13,2,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,1,0,0,0,0,1
4,4,6,8,13,8,13,7,11,7,13,13,22,13,23,1487.0,1.3045,23,0.9472,0.0006,-1,25,9,17,12,16,9,17,-1,22,9,8,10,20,2,6,-1,4,17,16,2,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0


In [27]:
Ytrain.head()

0    0
1    0
2    0
3    0
4    0
Name: QuoteConversion_Flag, dtype: int64

In [28]:
print(Xtest.shape)
print(Xtest.dtypes)
Xtest.head()

(173836, 595)
CoverageField11A        int64
CoverageField11B        int64
CoverageField1A         int64
CoverageField1B         int64
CoverageField2A         int64
                        ...  
GeographicField63_Y     int64
GeographicField64_CA    int64
GeographicField64_IL    int64
GeographicField64_NJ    int64
GeographicField64_TX    int64
Length: 595, dtype: object


,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_Q,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX
0,13,22,4,4,4,4,3,3,3,4,13,22,13,23,1487.0,1.3045,16,0.9364,0.0006,-1,25,9,18,13,18,9,18,-1,19,10,13,8,17,2,10,-1,20,19,18,25,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0
1,4,5,8,14,8,14,7,12,8,13,13,22,13,23,564.0,1.1886,11,0.9919,0.0038,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,17,...,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0
2,3,3,11,18,11,18,10,16,10,18,13,22,13,23,564.0,1.0670,15,0.8945,0.0038,-1,25,4,10,10,16,5,10,-1,11,11,18,5,13,2,13,-1,15,21,20,3,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0
3,5,9,14,22,15,22,13,20,22,25,13,22,13,23,1113.0,1.2665,21,0.8870,0.0004,-1,25,24,24,23,23,24,24,25,25,23,22,24,25,2,7,-1,13,11,14,9,...,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0
4,12,21,4,5,4,5,4,4,4,5,13,22,13,23,935.0,1.0200,25,0.9153,0.0007,-1,25,2,3,3,5,2,5,-1,7,6,6,2,6,15,23,-1,17,4,8,2,...,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0


In [29]:
#SMOTE==============================================================================
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Ytrain))#before smote
sm = SMOTE(sampling_strategy='float', ratio=0.5)#no of minority /majority
X_res, Y_res = sm.fit_resample(Xtrain, Ytrain)#after smote
print('Resampled dataset shape %s' % Counter(Y_res))#after smote
#new training set with x n y numbers changed #we can do tjis before we do all the previous model building

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


In [0]:

#CONSTRUCT DEFAULT DECISION TREE AND OBTAIN RESPECTIVE ACCURACY ==================
clf = DecisionTreeClassifier()
clf.fit(X_res, Y_res)
clf_predict_train=clf.predict(X_res)


print("accuracy Score (training) for Decision TreE:{0:6f}".format(clf.score(X_res,Y_res)))
print("Confusion Matrix for Decision Tree")
print(confusion_matrix(Y_res,clf_predict_train))
print("Classification report for Decision Tree")
print(classification_report(Y_res,clf_predict_train))




accuracy Score (training) for Decision TreE:1.000000
Confusion Matrix for Decision Tree
[[52738     0]
 [    0 26369]]
Classification report for Decision Tree
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     52738
           1       1.00      1.00      1.00     26369

    accuracy                           1.00     79107
   macro avg       1.00      1.00      1.00     79107
weighted avg       1.00      1.00      1.00     79107



In [0]:

Ypred_test = pd.DataFrame(clf.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(clf.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results1_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results1_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,1


In [0]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=10)
clf_random.fit(X_res, Y_res)
rand_parm=clf_random.best_params_
print(rand_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**rand_parm)
clf.fit(X_res,Y_res)
clf_predict_train = clf.predict(X_res)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (training) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_res,Y_res)))
print("Confusion Matrix after hypertuning for Decision Tree")
print(confusion_matrix(Y_res,clf_predict_train))
print("=== Classification Report after hypertuning for Decision Tree===")
print(classification_report(Y_res,clf_predict_train))

#run cross-validation on best hyperparameters, get auc score
clf_cv_score = cross_val_score(clf, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 80, 'min_samples_leaf': 3}
accuracy Score (training) after hypertuning for Decision Tree:0.946225
Confusion Matrix after hypertuning for Decision Tree
[[51364  1374]
 [ 2880 23489]]
=== Classification Report after hypertuning for Decision Tree===
              precision    recall  f1-score   support

           0       0.95      0.97      0.96     52738
           1       0.94      0.89      0.92     26369

    accuracy                           0.95     79107
   macro avg       0.95      0.93      0.94     79107
weighted avg       0.95      0.95      0.95     79107

=== All AUC Scores ===
[0.42457272 0.49337188 0.66467873 0.82862868 0.98341901 0.97486159
 0.95187977 0.5075195 ]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7286164848718897


In [0]:
Ypred_test = pd.DataFrame(clf.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(clf.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results2_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results2_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#Random Forest =============================================================
#Default mode
rfc = RandomForestClassifier()
rfc.fit(X_res, Y_res)
rfc_predict_train=rfc.predict(X_res)
print("accuracy Score (training) for RandomForest:{0:6f}".format(rfc.score(X_res,Y_res)))
print("Confusion Matrix for Random Forest:")
print(confusion_matrix(Y_res,rfc_predict_train))
print("classification report for Random Forest:")
print(classification_report(Y_res,rfc_predict_train))

accuracy Score (training) for RandomForest:0.994741
Confusion Matrix for Random Forest:
[[52727    11]
 [  405 25964]]
classification report for Random Forest:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00     52738
           1       1.00      0.98      0.99     26369

    accuracy                           0.99     79107
   macro avg       1.00      0.99      0.99     79107
weighted avg       0.99      0.99      0.99     79107



In [0]:
Ypred_test = pd.DataFrame(rfc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(rfc.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results3_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results3_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#Hyperparameter tuning for random forest
parameters={ 'n_estimators': range(50,100,10),'min_samples_split' : range(10,100,10),'min_samples_leaf': range(2,8,1)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=10)
rfc_random.fit(X_res, Y_res)
rand_parm_rfc=rfc_random.best_params_
print(rand_parm_rfc)

#contruct random forest using the best parameters
rfc= RandomForestClassifier(**rand_parm_rfc)
rfc.fit(X_res,Y_res)
rfc_predict_train = rfc.predict(X_res)
print("accuracy Score (training) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_res,Y_res)))
print("Confusion Matrix after hypertuning for Random Forest:")
print(confusion_matrix(Y_res,rfc_predict_train))
print("=== Classification Report after hypertuning for Random Forest:===")
print(classification_report(Y_res,rfc_predict_train))


#run cross-validation on best parameters, get auc score
rfc_cv_score = cross_val_score(rfc, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())


{'n_estimators': 50, 'min_samples_split': 90, 'min_samples_leaf': 3}
accuracy Score (training) after hypertuning for Random Forest:0.941585
Confusion Matrix after hypertuning for Random Forest:
[[52092   646]
 [ 3975 22394]]
=== Classification Report after hypertuning for Random Forest:===
              precision    recall  f1-score   support

           0       0.93      0.99      0.96     52738
           1       0.97      0.85      0.91     26369

    accuracy                           0.94     79107
   macro avg       0.95      0.92      0.93     79107
weighted avg       0.94      0.94      0.94     79107

=== All AUC Scores ===
[0.92325879 0.92769195 0.9279917  0.94968913 0.99918798 0.99950698
 0.99935781 0.9972523 ]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9654920793704724


In [0]:
Ypred_test = pd.DataFrame(rfc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(rfc.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results4_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results4_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
#Neural networks=============================================================
#Default mode
nn = MLPClassifier()
nn.fit(X_res, Y_res)
nn_predict_train=nn.predict(X_res)
print("accuracy Score (training) for Neural network:{0:6f}".format(nn.score(X_res,Y_res)))
print("Confusion Matrix for neural network:")
print(confusion_matrix(Y_res,nn_predict_train))
print("=== Classification Report for neural network:===")
print(classification_report(Y_res,nn_predict_train))

accuracy Score (training) for Neural network:0.764420
Confusion Matrix for neural network:
[[52503   235]
 [18401  7968]]
=== Classification Report for neural network:===
              precision    recall  f1-score   support

           0       0.74      1.00      0.85     52738
           1       0.97      0.30      0.46     26369

    accuracy                           0.76     79107
   macro avg       0.86      0.65      0.66     79107
weighted avg       0.82      0.76      0.72     79107



In [0]:
Ypred_test = pd.DataFrame(nn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(nn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results5_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results5_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [0]:
 #Hyperparameter tuning for neural network
parameters={ 'activation': ['relu','logistic'], 'max_iter': range(100,150,10)}
nn_random = RandomizedSearchCV(nn,parameters,n_iter=10)
nn_random.fit(X_res, Y_res)
rand_parm_nn=nn_random.best_params_
print(rand_parm_nn)

#contruct mlclassifier using the best parameters
nn = MLPClassifier(**rand_parm_nn)
nn.fit(X_res, Y_res)
nn_predict_train = nn.predict(X_res)
print("accuracy Score (training) after hypertuning for Neural networks:{0:6f}".format(nn.score(X_res, Y_res)))
print("Confusion Matrix after hypertuning for Neural network:")
print(confusion_matrix(Y_res,nn_predict_train))
print("=== Classification Report after hypertuning for Neural network:===")
print(classification_report(Y_res,nn_predict_train))

#run cross-validation on best parameters, get auc score
nn_cv_score = cross_val_score(nn, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(nn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Neural network: ",nn_cv_score.mean())
 

In [0]:
Ypred_test = pd.DataFrame(nn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(nn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results6_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results6_hw3.csv")
res.head()

In [11]:
#SVC=============================================================
#Default mode
svc = LinearSVC()
svc.fit(X_res, Y_res)
svc_predict_train=svc.predict(X_res)
print("accuracy Score (training) for svc:{0:6f}".format(svc.score(X_res, Y_res)))
print("Confusion Matrix for svc:")
print(confusion_matrix(Y_res,svc_predict_train))
print("=== Classification Report for svc:===")
print(classification_report(Y_res,svc_predict_train))

accuracy Score (training) for svc:0.629957
Confusion Matrix for svc:
[[27619 25119]
 [ 4154 22215]]
=== Classification Report for svc:===
              precision    recall  f1-score   support

           0       0.87      0.52      0.65     52738
           1       0.47      0.84      0.60     26369

    accuracy                           0.63     79107
   macro avg       0.67      0.68      0.63     79107
weighted avg       0.74      0.63      0.64     79107



In [14]:
Ypred_test = pd.DataFrame(svc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#Ypred_prob=pd.DataFrame(svc.predict_proba(Xtest))
#Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results7_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results7_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,1
3,9,0
4,10,0


In [15]:
#Hyperparameter tuning for svc
parameters={ 'max_iter': range(10,80,10),'loss':['hinge','squared_hinge']} #'decision_function_shape': ['ovo','ovr']
svc = LinearSVC()
svc_random = RandomizedSearchCV(svc,parameters,n_iter=10)
svc_random.fit(X_res, Y_res)
rand_parm_svc=svc_random.best_params_
print(rand_parm_svc)

#contruct svc  using the best parameters
svc = LinearSVC(**rand_parm_svc)
svc.fit(X_res, Y_res)
svc_predict_train = svc.predict(X_res)
print("accuracy Score (training) after hypertuning for svc:{0:6f}".format(svc.score(X_res, Y_res)))
print("Confusion Matrix after hypertuning for svc:")
print(confusion_matrix(Y_res,svc_predict_train))
print("=== Classification Report after hypertuning for svc:===")
print(classification_report(Y_res,svc_predict_train))

#run cross-validation on best parameters, get auc score
svc_cv_score = cross_val_score(svc, X_res, Y_res, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(svc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",svc_cv_score.mean())

{'max_iter': 50, 'loss': 'hinge'}
accuracy Score (training) after hypertuning for svc:0.680521
Confusion Matrix after hypertuning for svc:
[[49429  3309]
 [21964  4405]]
=== Classification Report after hypertuning for svc:===
              precision    recall  f1-score   support

           0       0.69      0.94      0.80     52738
           1       0.57      0.17      0.26     26369

    accuracy                           0.68     79107
   macro avg       0.63      0.55      0.53     79107
weighted avg       0.65      0.68      0.62     79107

=== All AUC Scores ===
[0.65356334 0.85187989 0.45211007 0.60981189 0.7233465  0.60620212
 0.59787297 0.66798681 0.76477459 0.93768171]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.6865229904311958


In [17]:
Ypred_test = pd.DataFrame(svc.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#Ypred_prob=pd.DataFrame(svc.predict_proba(Xtest))
#Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results8_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results8_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [18]:
#KNN=============================================================
#Default mode
knn = KNeighborsClassifier()
knn.fit( X_res, Y_res)
knn_predict_train=knn.predict(X_res)
print("accuracy Score (training) for knn:{0:6f}".format(knn.score( X_res, Y_res)))
print("Confusion Matrix for knn:")
print(confusion_matrix(Y_res,knn_predict_train))
print("=== Classification Report for knn:===")
print(classification_report(Y_res,knn_predict_train))

accuracy Score (training) for knn:0.823682
Confusion Matrix for knn:
[[45165  7573]
 [ 6375 19994]]
=== Classification Report for knn:===
              precision    recall  f1-score   support

           0       0.88      0.86      0.87     52738
           1       0.73      0.76      0.74     26369

    accuracy                           0.82     79107
   macro avg       0.80      0.81      0.80     79107
weighted avg       0.83      0.82      0.82     79107



In [19]:
Ypred_test = pd.DataFrame(knn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(knn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results9_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results9_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [21]:
#Hyperparameter tuning for knn
parameters={ 'n_neighbors': [3,5,7], 'leaf_size': [20,30,40]}
knn_random = RandomizedSearchCV(knn,parameters,n_iter=10)
knn_random.fit(X_res, Y_res)
rand_parm_knn=knn_random.best_params_
print(rand_parm_knn)

#contruct knn  using the best parameters
knn = KNeighborsClassifier(**rand_parm_knn)
knn.fit(X_res, Y_res)
knn_predict_train = knn.predict(X_res)
print("accuracy Score (training) after hypertuning for knn:{0:6f}".format(knn.score(X_res, Y_res)))
print("Confusion Matrix after hypertuning for knn:")
print(confusion_matrix(Y_res,knn_predict_train))
print("=== Classification Report after hypertuning for knn:===")
print(classification_report(Y_res,knn_predict_train))

#run cross-validation on best parameters, get auc score
knn_cv_score = cross_val_score(knn, X_res, Y_res, cv=8, scoring="roc_auc")
print("=== All AUC Scores ===")
print(knn_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - svc: ",knn_cv_score.mean())

{'n_neighbors': 3, 'leaf_size': 20}
accuracy Score (training) after hypertuning for knn:0.874360
Confusion Matrix after hypertuning for knn:
[[46280  6458]
 [ 3481 22888]]
=== Classification Report after hypertuning for knn:===
              precision    recall  f1-score   support

           0       0.93      0.88      0.90     52738
           1       0.78      0.87      0.82     26369

    accuracy                           0.87     79107
   macro avg       0.85      0.87      0.86     79107
weighted avg       0.88      0.87      0.88     79107

=== All AUC Scores ===
[0.30029924 0.11120574 0.11332012 0.13601019 0.23692622 0.24874904
 0.2316342  0.21731519]


=== Mean AUC Score ===
Mean AUC Score - svc:  0.1994324927506318


In [22]:
Ypred_test = pd.DataFrame(knn.predict(Xtest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

Ypred_prob=pd.DataFrame(knn.predict_proba(Xtest))
Ypred_prob.head()

pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results10_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results10_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [11]:
#STACKING MODELS =====================================================================
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using Neural networks,support vector machine, knn, RandomForest and Decision Tree Classifier\n")

models = [LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier() ]#we r giving models as arguments we can also give those parameters like grid_params
      #we are giving  data after smote and we did smote only for train test
Strain, Stest = stacking(models,                   
                           X_res, Y_res, Xtest,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using Neural networks,support vector machine, knn, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LinearSVC]
    fold  0:  [0.50131459]
    fold  1:  [0.69110583]
    fold  2:  [0.66155947]
    fold  3:  [0.39224312]
    ----
    MEAN:     [0.56155575] + [0.12153161]
    FULL:     [0.56155587]

model  1:     [RandomForestClassifier]
    fold  0:  [0.90868642]
    fold  1:  [0.90863124]
    fold  2:  [0.91186286]
    fold  3:  [0.90104167]
    ----
    MEAN:     [0.90755555] + [0.00398182]
    FULL:     [0.90755559]

model  2:     [DecisionTreeClassifier]
    fold  0:  [0.90539994]
    fold  1:  [0.89932750]
    fold  2:  [0.90094053]
    fold  3:  [0.89947411]
    ----
    MEAN:     [0.90128552] + [0.00245777]
    FULL:     [0.901

In [12]:
#STACKING - CONTRUCT A decision tree classifier MODEL==============================
model = DecisionTreeClassifier()
    
model = model.fit(Strain, Y_res)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using decision tree classifier: [%.8f]' % accuracy_score(Y_res, Ypred_train))
print("Confusion Matrix after STACKING for decision tree classifier :")
print(confusion_matrix(Y_res, Ypred_train))
print("=== Classification Report after STACKING for decision tree classifier===")
print(classification_report(Y_res, Ypred_train))

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#we are doing one layer with decision tree classifier

Final prediction score for ensemble methods using decision tree classifier: [0.90816236]
Confusion Matrix after STACKING for decision tree classifier :
[[52118   620]
 [ 6645 19724]]
=== Classification Report after STACKING for decision tree classifier===
              precision    recall  f1-score   support

           0       0.89      0.99      0.93     52738
           1       0.97      0.75      0.84     26369

    accuracy                           0.91     79107
   macro avg       0.93      0.87      0.89     79107
weighted avg       0.91      0.91      0.90     79107



,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [13]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.926251,0.073749
1,0.954861,0.045139
2,0.926251,0.073749
3,0.954861,0.045139
4,0.954861,0.045139


In [14]:
pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_DT_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_DT_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [15]:
#STACKING - CONTRUCT A GradientBoostingClassifier MODEL==============================
model = GradientBoostingClassifier()
    
model = model.fit(Strain, Y_res)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using GradientBoostingClassifier: [%.8f]' % accuracy_score(Y_res, Ypred_train))
print("Confusion Matrix after STACKING for GradientBoostingClassifier() :")
print(confusion_matrix(Y_res, Ypred_train))
print("=== Classification Report after STACKING for GradientBoostingClassifier()r===")
print(classification_report(Y_res, Ypred_train))

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#we are doing one layer with GradientBoostingClassifier()

Final prediction score for ensemble methods using GradientBoostingClassifier: [0.90816236]
Confusion Matrix after STACKING for GradientBoostingClassifier() :
[[52118   620]
 [ 6645 19724]]
=== Classification Report after STACKING for GradientBoostingClassifier()r===
              precision    recall  f1-score   support

           0       0.89      0.99      0.93     52738
           1       0.97      0.75      0.84     26369

    accuracy                           0.91     79107
   macro avg       0.93      0.87      0.89     79107
weighted avg       0.91      0.91      0.90     79107



,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [16]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.926244,0.073756
1,0.954852,0.045148
2,0.926244,0.073756
3,0.954852,0.045148
4,0.954852,0.045148


In [17]:
pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_GB_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_GB_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [18]:
#STACKING - CONTRUCT A random forest classfier MODEL==============================
model = RandomForestClassifier()
    
model = model.fit(Strain, Y_res)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using random forest: [%.8f]' % accuracy_score(Y_res, Ypred_train))
print("Confusion Matrix after STACKING for decision tree classifier :")
print(confusion_matrix(Y_res, Ypred_train))
print("=== Classification Report after STACKING for random forestr===")
print(classification_report(Y_res, Ypred_train))

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#we are doing one layer with random forest classifier

Final prediction score for ensemble methods using random forest: [0.90816236]
Confusion Matrix after STACKING for decision tree classifier :
[[52118   620]
 [ 6645 19724]]
=== Classification Report after STACKING for random forestr===
              precision    recall  f1-score   support

           0       0.89      0.99      0.93     52738
           1       0.97      0.75      0.84     26369

    accuracy                           0.91     79107
   macro avg       0.93      0.87      0.89     79107
weighted avg       0.91      0.91      0.90     79107



,QuoteConversion_Flag
0,0
1,0
2,0
3,0
4,0


In [19]:
Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
Ypred_prob.head()

,0,1
0,0.926592,0.073408
1,0.955093,0.044907
2,0.926592,0.073408
3,0.955093,0.044907
4,0.955093,0.044907


In [20]:
pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_RF_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_RF_hw3.csv")
res.head()

,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0


In [31]:
#STACKING - CONTRUCT A linear SVC MODEL==============================
model = LinearSVC()
    
model = model.fit(Strain, Y_res)
Ypred_train = model.predict(Strain)
print('Final prediction score for ensemble methods using linear SVC: [%.8f]' % accuracy_score(Y_res, Ypred_train))
print("Confusion Matrix after STACKING for linear SVC :")
print(confusion_matrix(Y_res, Ypred_train))
print("=== Classification Report after STACKING for linear SVC ===")
print(classification_report(Y_res, Ypred_train))

Ypred_test = pd.DataFrame(model.predict(Stest),columns=["QuoteConversion_Flag"])

Ypred_test.head()

#we are doing one layer linear SVC MOdel
#Ypred_prob=pd.DataFrame(model.predict_proba(Stest))
#Ypred_prob.head()
pd.concat([test_data["QuoteNumber"],Ypred_test],axis=1).to_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_SVC_hw3.csv",index=None)
res=pd.read_csv("/gdrive/My Drive/CIS508/Assignment-3/Results_SVC_hw3.csv")
res.head()

Final prediction score for ensemble methods using random forest: [0.90755559]
Confusion Matrix after STACKING for decision tree classifier :
[[51205  1533]
 [ 5780 20589]]
=== Classification Report after STACKING for random forestr===
              precision    recall  f1-score   support

           0       0.90      0.97      0.93     52738
           1       0.93      0.78      0.85     26369

    accuracy                           0.91     79107
   macro avg       0.91      0.88      0.89     79107
weighted avg       0.91      0.91      0.91     79107



,QuoteNumber,QuoteConversion_Flag
0,3,0
1,5,0
2,7,0
3,9,0
4,10,0
